In [3]:
import pandas as pd
import polars as pl

In [4]:
!pwd

/Users/ivustianiu/PycharmProjects/mlops-zoomcamp/project


In [43]:
df = pd.read_csv('output/clinvar_annotated.tsv.gz', delimiter='\t', na_values='-')
df.shape

(9058, 23)

In [44]:
dg = pl.read_csv('output/clinvar_annotated.tsv.gz', separator='\t', null_values='-')
dg.shape

(9058, 23)

In [45]:
# Metadata columns (must be excluded): variant_id, gene_name, ncbi_id, transcript_id, transcript_version
# Target column: label
# Atomic categorical columns: ref, alt, consequence, impact
# Compound categorical column (must be split into ref_aa and alt_aa): amino_acid_change

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9058 entries, 0 to 9057
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   variant_id          9058 non-null   object 
 1   ref                 9058 non-null   object 
 2   alt                 9058 non-null   object 
 3   clnsig              9058 non-null   object 
 4   gnomad_ac           4034 non-null   float64
 5   gnomad_af           4034 non-null   float64
 6   gnomad_an           4034 non-null   float64
 7   gnomad_hom          4034 non-null   float64
 8   revel               8987 non-null   float64
 9   ds_ag               8943 non-null   float64
 10  ds_al               8943 non-null   float64
 11  ds_dg               8943 non-null   float64
 12  ds_dl               8943 non-null   float64
 13  amino_acid_change   8308 non-null   object 
 14  biotype             9058 non-null   object 
 15  canonical           9058 non-null   bool   
 16  conseq

In [48]:
df['clnsig'].value_counts(normalize=True)

clnsig
LB    0.473283
LP    0.248620
B     0.221130
P     0.056966
Name: proportion, dtype: float64

In [60]:
aa_cnt = df['amino_acid_change'].value_counts(dropna=False)
aa_cnt

amino_acid_change
NaN    750
R/Q    294
R/H    271
A/T    268
A/V    253
      ... 
S/*      1
R/*      1
E/*      1
W/*      1
H        1
Name: count, Length: 174, dtype: int64

In [72]:
for i in (1, 10, 100, 750):
    print(f'Count <= {i}: {(aa_cnt <= i).sum()} entries')

Count <= 1: 7 entries
Count <= 10: 35 entries
Count <= 100: 153 entries
Count <= 750: 174 entries


In [74]:
def report_constant_columns(df):
    for col in df.columns:
        if df[col].nunique() == 1:
            print(col)


report_constant_columns(df)            

biotype


In [79]:
df['canonical'].value_counts(dropna=False)

canonical
True     4903
False    4155
Name: count, dtype: int64

In [80]:
df['impact'].value_counts()

impact
MODERATE    8139
MODIFIER     726
LOW          106
HIGH          87
Name: count, dtype: int64

In [85]:
df['consequence'].value_counts(ascending=True)

consequence
splice_polypyrimidine_tract_variant&intron_variant                             1
splice_donor_region_variant&intron_variant                                     1
splice_region_variant&intron_variant                                           2
stop_gained&splice_region_variant                                              2
splice_region_variant&5_prime_UTR_variant                                      3
splice_region_variant&synonymous_variant                                       4
splice_acceptor_variant                                                        4
splice_polypyrimidine_tract_variant&splice_region_variant&intron_variant       6
splice_donor_variant                                                           7
stop_gained                                                                    7
3_prime_UTR_variant                                                           26
5_prime_UTR_variant                                                           53
start_lost      

In [86]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Example DataFrame
data = {
    'col': [['a', 'b'], ['b', 'c'], ['a'], ['c'], ['a', 'c'], ['b']]
}
df = pd.DataFrame(data)

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'col' column
binary_matrix = mlb.fit_transform(df['col'])

# Create a DataFrame from the binary matrix
binary_df = pd.DataFrame(binary_matrix, columns=mlb.classes_)

# Concatenate the new binary DataFrame with the original DataFrame
df = pd.concat([df, binary_df], axis=1)

# Drop the original 'col' column
df = df.drop(columns=['col'])

print(df)


   a  b  c
0  1  1  0
1  0  1  1
2  1  0  0
3  0  0  1
4  1  0  1
5  0  1  0


In [19]:
df[~df['Consequence'].isin(
    ['intron_variant', 'upstream_gene_variant', 'downsream_gene_variant', 'regulatory_region_variant']
)].head(1).T

,17
variant_id,chr1_1273745_C_T
label,LB
Allele,T
Consequence,5_prime_UTR_variant
IMPACT,MODIFIER
SYMBOL,UBE2J2
Gene,ENSG00000160087
Feature_type,Transcript
Feature,ENST00000349431.11
BIOTYPE,protein_coding
